In [3]:
import en_core_web_md
from itertools import product
from collections import defaultdict



In [5]:
from common import Relation
nlp = en_core_web_md.load()

In [6]:
doc = nlp("Farley attorney Stephen Greiner contended during Wednesday 's 90-minute hearing that the pill ` ` is a device by which the board of directors of West Point says nobody can buy more than 10 percent.")

In [8]:
doc

Farley attorney Stephen Greiner contended during Wednesday 's 90-minute hearing that the pill ` ` is a device by which the board of directors of West Point says nobody can buy more than 10 percent.

In [9]:
list(doc.noun_chunks)

[Farley attorney Stephen Greiner,
 Wednesday 's 90-minute hearing,
 the pill,
 a device,
 the board,
 directors,
 West Point,
 nobody,
 more than 10 percent]

In [11]:
from extract import  ProcessAnnotatedData, RelationSentence

In [12]:
import pickle

In [71]:
with open("test_data.pkl", 'rb') as f:
    data = pickle.load(f)


In [14]:
work_for = []
for sent, rel_sent in data.i2sentence.items():
    for (a0, rel, a1) in data.i2relations[sent]:
        if rel == 'Work_For':
            work_for.append((rel_sent, (a0, rel, a1)))

In [15]:
for rel_sent, (a0, rel, a1) in work_for:
    print("#"*25)
    print(f"a0: {a0}, rel: : {rel}, a1: : {a1}")
    for chunk in (list(rel_sent.analyzed.noun_chunks)):
        print(chunk)

#########################
a0: W. Dale Nelson, rel: : Work_For, a1: : The Associated Press
W. Dale Nelson
the White House
The Associated Press
#########################
a0: Ralph K. Winter, rel: : Work_For, a1: : 2nd U.S. Circuit Court of Appeals
Judge Ralph K. Winter
the 2nd U.S. Circuit Court
Appeals
New York City
Judge Kenneth Starr
the U.S. Circuit Court
Appeals
the District
Columbia
the source
who
condition
anonymity
#########################
a0: Kenneth Starr, rel: : Work_For, a1: : U.S. Circuit Court of Appeals
Judge Ralph K. Winter
the 2nd U.S. Circuit Court
Appeals
New York City
Judge Kenneth Starr
the U.S. Circuit Court
Appeals
the District
Columbia
the source
who
condition
anonymity
#########################
a0: Wright, rel: : Work_For, a1: : University of Texas
Wright
Texas
Nixon 's lawyers
the Watergate special prosecutor
the White House tapes
#########################
a0: Winter, rel: : Work_For, a1: : Yale University
Winter
a former Yale University law professor
who
the b

In [45]:
pred = defaultdict(list)
correct = 0
wrong= []
for rel_sent, (a0, rel, a1) in work_for:
    for chunk in (list(rel_sent.analyzed.noun_chunks)):
        # print(chunk)
        for (cand_per,cand_org) in product(rel_sent.entities['PER'], rel_sent.entities['ORG']):
            # str(chunk)
            if cand_org['text'] in chunk.text and cand_per['text'] in chunk.text:
                pred[rel_sent.idx].append((cand_per, cand_org))
                print("*"*25)
                print(f"gold: {a0} | pred: {cand_per['text']}")
                print(f"gold: {a1} | pred: {cand_org['text']}")
                print(f"original chunk: {chunk.text} ")
                if a0 == cand_per['text'] and a1==cand_org['text']:
                    correct+=1
                else:
                    wrong.append([a0,cand_per['text'], a1, cand_org['text']])
            #

*************************
gold: Michael D. Papagiannis | pred: Michael D. Papagiannis
gold: Boston University | pred: Boston University
original chunk: Boston University 's Michael D. Papagiannis 
*************************
gold: Mark Moseley | pred: Mark Moseley
gold: National Football League | pred: National Football League
original chunk: retired National Football League placekicker Mark Moseley 
*************************
gold: Fred Eckhard | pred: Fred Eckhard
gold: U.N. | pred: U.N.
original chunk: U.N. financial spokesman Fred Eckhard 
*************************
gold: Javier Perez de Cuellar | pred: Javier Perez de Cuellar
gold: U.N. | pred: U.N
original chunk: U.N. Secretary-General Javier Perez de Cuellar 
*************************
gold: Sohn Ho Young | pred: Sohn Ho Young
gold: Korean CIA | pred: CIA
original chunk: high-ranking Korean CIA official Sohn Ho Young 
*************************
gold: J.H. Berman | pred: Berman
gold: Supreme Court | pred: Cape Town Supreme Court
origin

In [42]:
len(pred)

17

In [43]:
len(work_for)

109

In [44]:
correct


13

In [54]:
not_work_for = []
for sent, rel_sent in data.i2sentence.items():
    if all([rel != 'Work_For' for (a0, rel, a1) in data.i2relations[sent]]):
        for (a0, rel, a1) in data.i2relations[sent]:
                not_work_for.append((rel_sent, (a0, rel, a1)))

In [52]:
# not_work_for

In [57]:
pred_wrong = defaultdict(list)
false_pos = 0
for rel_sent, (a0, rel, a1) in not_work_for:
    for chunk in (list(rel_sent.analyzed.noun_chunks)):
        # print(chunk)
        for (cand_per,cand_org) in product(rel_sent.entities['PER'], rel_sent.entities['ORG']):
            # str(chunk)
            if cand_org['text'] in chunk.text and cand_per['text'] in chunk.text and cand_per['text'] not in cand_org['text'] and cand_org['text'] not in cand_per['text']:
                pred_wrong[rel_sent.idx].append((cand_per, cand_org))
                print("*"*25)
                print(f"rel: {rel}")
                print(f"gold: {a0} | pred: {cand_per['text']}")
                print(f"gold: {a1} | pred: {cand_org['text']}")
                print(f"original chunk: {chunk.text} ")
                if a0 == cand_per['text'] and a1==cand_org['text']:
                    false_pos+=1
            #

*************************
rel: Live_In
gold: Bill Gordon | pred: Bill Gordon
gold: Bingham County | pred: Bingham County Sheriff
original chunk: Bingham County Sheriff 's Lt. Bill Gordon 
*************************
rel: Live_In
gold: Adam Urbanski | pred: Adam Urbanski
gold: Rochester | pred: Rochester
original chunk: Rochester teacher union leader Adam Urbanski 
*************************
rel: Live_In
gold: Peter McWalters | pred: Peter McWalters
gold: Rochester | pred: Rochester
original chunk: Rochester 's Peter McWalters 
*************************
rel: Live_In
gold: Carl Rubin | pred: Carl Rubin
gold: U.S. | pred: U.S. District Court
original chunk: U.S. District Court Judge Carl Rubin 


In [78]:
all_relation = []
for sent, rel_sent in data.i2sentence.items():
        for (a0, rel, a1) in data.i2relations[sent]:
                all_relation.append((rel_sent, (a0, rel, a1)))

pred_gen = defaultdict(list)
correct_ = 0
fuzzy_correct = 0
wrong_rel=0
for rel_sent, (a0, rel, a1) in all_relation:
    for chunk in (list(rel_sent.analyzed.noun_chunks)):
        for (cand_per,cand_org) in product(rel_sent.entities['PER'], rel_sent.entities['ORG']):
            # str(chunk)
            if cand_org['text'] in chunk.text and cand_per['text'] in chunk.text and cand_per['text'] not in cand_org['text'] and cand_org['text'] not in cand_per['text']:
                pred_gen[rel_sent.idx].append((cand_per, cand_org))
                # print("*"*25)
                # print(f"rel: {rel}")
                # print(f"gold: {a0} | pred: {cand_per['text']}")
                # print(f"gold: {a1} | pred: {cand_org['text']}")
                # print(f"original chunk: {chunk.text} ")
                if a0 == cand_per['text'] and a1==cand_org['text'] and rel == 'Work_For':
                    correct_ +=1
                    fuzzy_correct+=1
                elif (a0 in cand_per['text'] or cand_per['text'] in a0) and \
                     (a1 in cand_org['text'] or cand_org['text'] in a1) and \
                      rel == 'Work_For':
                    fuzzy_correct+=1
                    print("*"*25)
                    print(f"rel: {rel}")
                    print(f"gold: {a0} | pred: {cand_per['text']}")
                    print(f"gold: {a1} | pred: {cand_org['text']}")
                    print(f"original chunk: {chunk.text} ")

                elif rel !='Work_For':
                    wrong_rel +=1
                    # print("*"*25)
                    # print(f"rel: {rel}")
                    # print(f"gold: {a0} | pred: {cand_per['text']}")
                    # print(f"gold: {a1} | pred: {cand_org['text']}")
                    # print(f"original chunk: {chunk.text} ")

                else:
                    print("*"*25)
                    print(f"rel: {rel}")
                    print(f"gold: {a0} | pred: {cand_per['text']}")
                    print(f"gold: {a1} | pred: {cand_org['text']}")
                    print(f"original chunk: {chunk.text} ")

*************************
rel: Work_For
gold: Hernandez Galicia | pred: Hernandez Galicia 's
gold: Mexican Oil Workers Union | pred: Mexican Oil Workers Union
original chunk: Hernandez Galicia 's allegedly corrupt Mexican Oil Workers Union 
*************************
rel: Work_For
gold: Will Carlson. | pred: Will Carlson
gold: Michigan State | pred: Michigan State
original chunk: Michigan State horticulture Professor Will Carlson 
*************************
rel: Work_For
gold: Mike Ferris. | pred: Mike Ferris
gold: Forest Service | pred: Forest Service
original chunk: Forest Service spokesman Mike Ferris 
*************************
rel: Work_For
gold: Margie Clack. | pred: Margie Clack
gold: Forest Service | pred: Forest Service
original chunk: Forest Service spokeswoman Margie Clack 
*************************
rel: Work_For
gold: Oliver North. | pred: Oliver North
gold: National Security Council | pred: National Security Council
original chunk: former National Security Council aide Oliver

In [73]:
correct_


12

In [74]:
fuzzy_correct

21

In [75]:
wrong

[['Javier Perez de Cuellar', 'Javier Perez de Cuellar', 'U.N.', 'U.N'],
 ['Sohn Ho Young', 'Sohn Ho Young', 'Korean CIA', 'CIA'],
 ['J.H. Berman', 'Berman', 'Supreme Court', 'Cape Town Supreme Court'],
 ['Kevin Kelly', 'Kevin Kelly', 'Southern California Edison', 'Edison'],
 ['Lindsay Ellis',
  'Lindsay Ellis',
  'Superior Court',
  'Maricopa County Superior Court']]

In [76]:
wrong_rel

11

In [3]:
import pickle
##########################
train_vectorized = load_from_pickle("train.vec")
test_vectorized = load_from_pickle("test.vec")
train_labels = load_from_pickle("train_label.vec")
###########################

In [2]:
import pickle
def load_from_pickle(f_name):
    with open(f_name, 'rb') as f:
        data = pickle.load(f)
    return data

In [ ]:
def predict(test_vectors, op_relations):
    pred = model.predict(test_vectors)
    res = defaultdict(list)
    for i, (idx, person, org, sentence) in enumerate(op_relations):
        if pred[i] == "1":
            res[idx].append(Relation(person[TEXT], org[TEXT], sentence))
    return res


In [ ]:
from xgboost import XGBClassifier
for n_estimators_val in [100, 200, 400, 600, 800, 1000]:
    for scale_pos_weight_val in [1,10,20,30,50,60,70,80,90, 99]:
            xgboost = XGBClassifier(n_estimators=n_estimators_val,
                          scale_pos_weight=scale_pos_weight_val)
            xgboost.fit(train_vectorized, train_labels)
            pred = xgboost.predict(test_vectorized)

